#P0. Introducción -PLN

---

En la creación de redes neuronales necesitamos dos tipos de IA, para reconocer patrones o generar nuevos:

*   Las que no tienen memoria, identifica el patrón y ya!...ejemplo las de visión artificial
*   Las de memoria corta (Long Short Term Memory)...PLN
*   Las que requieren mucha memoria (aprenden casi todo...BERT)...PLN y visión artificial.



**Caso de estudio: generación de texto**

---


Cualquier dato que se necesite procesar (sonido, imágenes, texto) primero debe ser convertido en un tensor numérico, un paso llamado “vectorización” (One-hot Encoding y WordEmbedding) de datos (y en nuestro ejemplo previamente las letras deben ser pasadas a valores numéricos 

Para este ejemplo usaremos “*Character level language model*” propuesto por Andrej Karpathy en su artículo "*The Unreasonable Effectiveness of Recurrent Neural Networks*"(y parcialmente basado en su implementado en el tutorial "*Generate text with an RNN*" de la web de TensorFlow:

Consiste en darle a la RNN una palabra y se le pide que modele la distribución de probabilidad del siguiente carácter que le correspondería a la secuencia de caracteres anteriores:

Como ejemplo, supongamos que solo tenemos un vocabulario de cuatro letras posibles [“a”,”h”,”l”,”o”], y queremos entrenar a una RNN en la secuencia de entrenamiento “hola”. Esta secuencia de entrenamiento es, de hecho, una fuente de 3 ejemplos de entrenamiento por separado: La probabilidad de “o” debería ser verosímil dada el contexto de “h”, “l” debería ser verosímil en el contexto de “ho”, y finalmente “a” debería ser también verosímil dado el contexto de “hol”.


---
*   https://unipython.com/generacion-de-textos-con-inteligencia-artificial/
*    https://medium.com/analytics-vidhya/solution-to-tensorflow-2-not-using-gpu-119fb3e04daa (https://www.youtube.com/watch?v=kaQCdv46OBA&ab_channel=JeffHeaton)

#Ejemplo : generar texto de cuentos, usando Keras

##P0. importar librerias

In [16]:
!pip install tensorflow-gpu

In [2]:
import tensorflow as tf
import numpy as np
import os
import time
import sys

uso de GPU para entrenar en tensorflow

---
https://medium.com/analytics-vidhya/solution-to-tensorflow-2-not-using-gpu-119fb3e04daa


In [18]:
print("Version: ", tf.__version__)
#print("Eager mode: ", tf.executing_eagerly())
print("GPU esta", "disponible" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")
print("Dispositivos disponibles: ", tf.config.list_physical_devices())

Version:  2.7.0
GPU esta disponible
Dispositivos disponibles:  [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [19]:
#tf.device('/gpu:0') #activando la CPU
tf.device('/GPU:0') #activando la GPU 

##P0. Descarga y preprocesado de los datos

In [3]:
fileDL= tf.keras.utils.get_file('regueton bien sucio.txt','https://raw.githubusercontent.com/iFePax/DeepLearning/main/Datasets/regueton%20bien%20sucio.txt')
texto = open(fileDL, 'rb').read().decode(encoding='utf-8')
texto = texto.lower()

98304/85016 [==================================] - 0s 0us/step


##P1. entendiendo el texto

Normalizando el texto

In [4]:
print("[!] Pasando todo el texto a minuscula")
texto_min =texto.lower()
time.sleep(1)
print("[!] Quitando los caracteres especiales del texto")
texto_editado = texto_min.replace('\n', '').replace('\r','\n').replace('(','').replace(')','').replace('!','').replace('1','').replace('7','').replace('-','').replace('?','').replace('«','').replace('»','').replace('¿','').replace('¡','').replace('--','')
texto_final=texto_editado.replace('\n', '')
time.sleep(1)
print("Hecho !!")

[!] Pasando todo el texto a minuscula
[!] Quitando los caracteres especiales del texto
Hecho !!


In [5]:
print('el texto tiene longitud de:{} caracteres'. format(len(texto_final)))
vocab = sorted(set(texto_final))
print('el texto esta compuesto de estos :{} caracteres'. format(len(vocab)))
print(vocab)

el texto tiene longitud de:77199 caracteres
el texto esta compuesto de estos :52 caracteres
[' ', "'", ',', '.', '/', '0', '2', '3', '4', '5', '6', '8', '9', ':', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'á', 'é', 'í', 'ñ', 'ó', 'ú', 'ü', 'ō', 'е', '’']


##P2. pasar el texto a números

---
as redes neuronales solo procesan valores numéricos, no letras, por tanto tenemos que traducir los caracteres a representación numérica. Para ello crearemos dos “tablas de traducción”: una de caracteres a números y otra de números a caracteres

In [6]:
char2idx = {u:i for i, u in enumerate(vocab)} # asignamos un número a cada vocablo
idx2char = np.array(vocab)
#-----------revisando las conversiones
#for char,_ in zip(char2idx, range(len(vocab))):
#    print(' {:4s}: {:3d},'.format(repr(char),char2idx[char]))

#pasamos todo el texto a números
texto_como_entero= np.array([char2idx[c] for c in texto_final])
print('texto: {}'.format(repr(texto_final[:100])))
print('{}'.format(repr(texto_como_entero[:100])))

texto: '[darell]este es el verdadero remix, baby[ozuna]paso muchas noches pensándoteyo no sé ni cómo, ni cuá'
array([14, 19, 16, 33, 20, 27, 27, 15, 20, 34, 35, 20,  0, 20, 34,  0, 20,
       27,  0, 37, 20, 33, 19, 16, 19, 20, 33, 30,  0, 33, 20, 28, 24, 39,
        2,  0, 17, 16, 17, 40, 14, 30, 41, 36, 29, 16, 15, 31, 16, 34, 30,
        0, 28, 36, 18, 23, 16, 34,  0, 29, 30, 18, 23, 20, 34,  0, 31, 20,
       29, 34, 42, 29, 19, 30, 35, 20, 40, 30,  0, 29, 30,  0, 34, 43,  0,
       29, 24,  0, 18, 46, 28, 30,  2,  0, 29, 24,  0, 18, 36, 42])


##P3. preparar los datos para ser usados en la RNN

In [7]:
char_dataset= tf.data.Dataset.from_tensor_slices(texto_como_entero)
#cantidad de secuencia de caracteres
secu_length=150
#creamos secuencias de maximo 100 caractereres
secuencias= char_dataset.batch(secu_length+1, drop_remainder=True)
for item in secuencias.take(10):
  print(repr(''.join(idx2char[item.numpy()])))

'[darell]este es el verdadero remix, baby[ozuna]paso muchas noches pensándoteyo no sé ni cómo, ni cuándo fuepero solo sé que yo recordécómo te lo hacía '
"yo aquella vezy yo no puedo seguir solo, pero séque te botéde mi vida te boté, y te botéte di banda y te solté, yo te soltépa'l carajo usted se fue, y "
'usted se fuede mi vida te boté, yo te boté, yehyeh, mami[bad bunny]baby, la vida es un cicloy lo que no sirve ya no lo recicloasí que de mi vida muévet'
"eque si te lo meto, es pa' recordar un tbt, yehya yo me cansé de tus mentira'ahora hay una más dura que me tiratodo tiene su final, todo expiratú eres "
"pasado y el pasado nunca viraarranca pa'l carajo, mi cuerpo no te necesitalo que pide es un perreo sucio en la placitano creo que lo nuestro se repitad"
"ale, prende un phillie, deja uno ready pa' ahorita, yehodio saber que en ti una vez más yo confiéodio todos los te amo que mil veces te texteébaby, mej"
"or que tú, ahora tengo como die'lo nuestro iba en bugatti y te quedaste a p

###P3.1 separar los datos en agrupamientos (batches)

In [8]:
#funcion para obtener el conjunto de datos de trainning
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text= chunk[1:]
  return input_text, target_text

dataset  = secuencias.map(split_input_target)
#el dataset contiene un conjunto de parejas de secuencia de texto
#(con la representación numérica de los caracteres), donde el 
#primer componente de la pareja contiene un paquete con una secuencia 
#de 100 caracteres del texto original y la segunda su correspondiente salida, 
#también de 100 caracteres. )
for input_example, target_example in dataset.take(1):
  print('input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print('Target data: ', repr(''.join(idx2char[target_example.numpy()])))

input data:  '[darell]este es el verdadero remix, baby[ozuna]paso muchas noches pensándoteyo no sé ni cómo, ni cuándo fuepero solo sé que yo recordécómo te lo hacía'
Target data:  'darell]este es el verdadero remix, baby[ozuna]paso muchas noches pensándoteyo no sé ni cómo, ni cuándo fuepero solo sé que yo recordécómo te lo hacía '


In [9]:
#imprimimos el tensor del dataset
print(dataset)
#Hyper-Parametros para entrenamiento  de una rede neuronal 
#   -los datos se agrupan en batch
BATCH_SIZE= 64
#    -Tamaño de memoria disponible 
BUFFER_SIZE=10000
dataset= dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
print (dataset)
#En el tensor dataset disponemos los datos de entrenamiento
#con agrupamienttos (batches) compuestos de 64 parejas de secuencias 
#de 100 integers de 64 bits que representan el carácter correspondiente 
#en el vocabulario.

<MapDataset shapes: ((150,), (150,)), types: (tf.int64, tf.int64)>
<BatchDataset shapes: ((64, 150), (64, 150)), types: (tf.int64, tf.int64)>


##P4.Construcción del modelo RNN

---
Para construir el modelo usaremos tf.keras.Sequential. Usaremos una versión mínima de RNN, que contenga solo una capa LSTM y 3 capas.


In [10]:
#como es un problema de clasificación estándar 
#para el que debemos definir la función de Lossy el optimizador.
def lossy(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

def create_model(vocab_size, embedding_dim, rnn_units, batch_size):
  #creando el modelo
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                         return_sequences=True,
                         stateful=True,
                         recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)                               
  ])
  #En cuanto al optimizador usaremos tf.keras.optimizers.Adam 
  #con los argumentos por defecto del optimizador Adam. 
  model.compile(optimizer='adam',
              loss=lossy,
              metrics=['accuracy'])
  return model
vocab_size= len(vocab)
#dimensiones de los vectores que tendrá la capa.
embedding_dim= 256
#cantidad de neuronas
rnn_units=1024
#creamos nuestra red neuronal RNN
model=create_model(vocab_size   =vocab_size,
                  embedding_dim =embedding_dim,
                  rnn_units     =rnn_units,
                  batch_size    =BATCH_SIZE)
#summary()para visualizar la estructura del modelo
model.summary()
#resultados=  -La capa LSTM consta más de 5 millones de parametros)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           13312     
                                                                 
 lstm (LSTM)                 (64, None, 1024)          5246976   
                                                                 
 dense (Dense)               (64, None, 52)            53300     
                                                                 
Total params: 5,313,588
Trainable params: 5,313,588
Non-trainable params: 0
_________________________________________________________________


###P4.1 Creando chekpoints

---
una técnica de tolerancia de fallos para procesos cuyo tiempo de ejecución es muy largo. La idea es guardar una instantánea del estado del sistema periódicamente para recuperar desde ese punto la ejecución en caso de fallo del sistema.

---
los crearemos en google drive para mejorar la capacidad de reentrenamiento de la red


In [11]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [12]:
checkpoint_dir='/content/gdrive/MyDrive/CheckPoints'#direccion de la carpeta checkpoint
checkpoint_prefix= os.path.join(checkpoint_dir,"cp_{epoch:04d}.ckpt")


cp_callback=tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix,
                                               monitor='loss',
                                               verbose=1,
                                               save_weights_only=True,
                                               save_best_only=True,
                                               mode='auto')


###P4.2 entrenando

####P4.2a entrenando para usar chekpoints

In [ ]:
EPOCHS=1000 #Segun la cantidad de epoch que queramos seguir entrenando
history=model.fit(dataset, 
                  epochs=EPOCHS, 
                  verbose=1,
                  callbacks=[cp_callback])

Epoch 1/1000
5/7 [====================>.........] - ETA: 20s - loss: 3.6994 - accuracy: 0.1191


#####4.2a-1 entrenando desde un checkpoint

---
Desde la carpeta que optamos guardar los checkpoints

*   el archivo .data es el archivo que contiene nuestras variables de entrenamiento y vamos a ir tras él.
*   el archivo checkpoint, simplemente mantiene un registro de los últimos archivos de punto de control guardados




In [ ]:
#creamos un modelo con iguales caracteristicas al 1° modelo
model2=create_model(vocab_size   =vocab_size,
                  embedding_dim =embedding_dim,
                  rnn_units     =rnn_units,
                  batch_size    =BATCH_SIZE)

#buscamos el ultimo checkpoint de entrenamiento
latest = tf.train.latest_checkpoint(checkpoint_dir)
print(latest)

/content/gdrive/MyDrive/CheckPoints(generando texto de cuentos)/cp_0281.ckpt


In [ ]:
# cargamos los pesos al nuevo modelo (estos valores tienes una variación de un 10%)
model2.load_weights(latest)
# continuamos el entrenamiento desde el checkpoint en que quedamos
history_V2=model2.fit(dataset, 
                    epochs=800, 
                    verbose=1,
                    callbacks=[cp_callback])

Epoch 1/800
57/57 [==============================] - ETA: 0s - loss: 0.1143 - accuracy: 0.9766
Epoch 00001: loss improved from inf to 0.11427, saving model to /content/gdrive/MyDrive/CheckPoints(generando texto de cuentos)/cp_0001.ckpt
57/57 [==============================] - 22s 246ms/step - loss: 0.1143 - accuracy: 0.9766
Epoch 2/800
57/57 [==============================] - ETA: 0s - loss: 0.1133 - accuracy: 0.9769
Epoch 00002: loss improved from 0.11427 to 0.11328, saving model to /content/gdrive/MyDrive/CheckPoints(generando texto de cuentos)/cp_0002.ckpt
57/57 [==============================] - 15s 248ms/step - loss: 0.1133 - accuracy: 0.9769
Epoch 3/800
57/57 [==============================] - ETA: 0s - loss: 0.1134 - accuracy: 0.9767
Epoch 00003: loss did not improve from 0.11328
57/57 [==============================] - 15s 243ms/step - loss: 0.1134 - accuracy: 0.9767
Epoch 4/800
57/57 [==============================] - ETA: 0s - loss: 0.1146 - accuracy: 0.9767
Epoch 00004: loss

####P4.2b entrenando con tensorboard (opcional)

#####Activando TENSORBOARD 

---
(DASHBOARD para ver el proceso de entrenamiento)

In [ ]:
# You can change the directory name
LOG_DIR = '/content/drive/MyDrive/CheckPoints(generando texto de cuentos)'

!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

import os
if not os.path.exists(LOG_DIR):
  os.makedirs(LOG_DIR)
  
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR))

get_ipython().system_raw('./ngrok http 6006 &')

!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

In [ ]:
tbCallBack = tf.keras.callbacks.TensorBoard(log_dir=LOG_DIR, 
                         histogram_freq=1,
                         write_graph=True,
                         write_grads=True,
                         batch_size=BATCH_SIZE,
                         write_images=True)

#####Fit

In [ ]:

#model.fit(X, y, epochs=50, batch_size=64, callbacks=callbacks_list)

##P5. Generando texto nuevo usando la RNN

In [ ]:
#creamos un modelo tomando como base el ultimo checkpoint
model = create_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))##tomamos el ultimo checkpoint
model.build(tf.TensorShape([1,None]))

In [ ]:
#funcion para generar texto
def generate_text(model, start_string):
  #definimos cuantos tensores/cantidad de texto generaremos
  num_generate=500
  #convertimos el texto en números
  input_eval=[char2idx[s] for s in start_string]
  input_eval= tf.expand_dims (input_eval,0)
  text_generated = []

  temperature = 0.3  #(0.0 a  1) entre más alta la temperatura más creatividad al modelo, pero tambien más errores ortograficos.
  model.reset_states() #bucle para generar caracteres, mediante predicciones
  for i in range(num_generate):
    predictions = model(input_eval)
    predictions = tf.squeeze(predictions, 0)
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
    input_eval= tf.expand_dims([predicted_id],0)
    text_generated.append (idx2char[predicted_id])
  
  return (start_string+ ''.join(text_generated))


###P5.1 generando texto 

In [ ]:
text = input('Ingresa cualquier texto que se te ocurra: ').lower()
print('\n\t[!]Generando texto\n')
time.sleep(1)
print(generate_text(model, start_string=text))


Ingresa cualquier texto que se te ocurra:  Vino entonces  a  toda  prisa

	[!]Generando texto

 vino entonces  a  toda  prisa  se caso  con igraine;  y por  ende  declaro  que  no  es bastardo.  y contra  quien  diga  que  no, sera rey y vencermosa  hijo  de  rey,  y  por  ende,  la  costumbre  del  rey  lot.  y  fue  muy  contento  de  su  se los      de lucio  mas de  cien mil.  y  despues  el rey  arturo  hizo examinar todos  los  cuerpos  muertos,  y  enterrar  a  los  que  eran  de  su  sequito  y  probar  sobre  su  honra;  y  en  especial  que  no vergo  sir  gawain  que  seria  gran  vergüenza   para  ellos  i


##P6.exportando modelo

---
Guardamos y Serializamos el Modelo (con esto ya podemos vender nuestro modelo de predicción de texto según lo aprendido por nuestra RNN).


In [ ]:
from keras.models import model_from_json
import os
dir_export= '/content/gdrive/MyDrive/Models(RNN)' #Ruta del directorio el cual vamos a guardar el modelo
#dir_export= os.path.join(dir_drive)
# Serializamos el modelo en forma JSON
model_json = model.to_json()
with open(os.path.join(dir_export,'RNN_el_rey_arturo_y_los_caballeros_de_la_mesa_redonda_json.json'), 'w') as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights(os.path.join(dir_export,'RNN_el_rey_arturo_y_los_caballeros_de_la_mesa_redonda_pesos.hdf5'))
model.save(os.path.join(dir_export,'RNN_el_rey_arturo_y_los_caballeros_de_la_mesa_redonda_model.h5'))
print("modelo salvado en Drive de google")

modelo salvado en Drive de google


##P7.Cargando un modelo serializado

###P7.1 descargamos el modelo usando wget

Ponemos la url del

In [ ]:
!wget 'https://github.com/2HenryCardenas1/DeepLearning/blob/main/Jupyter/PLN(Procesamiento_de_lenguajes_naturales)/Models/RNN_el_rey_arturo_y_los_caballeros_de_la_mesa_redonda_model.h5?raw=true' -O el_rey_arturo_y_los_caballeros_de_la_mesa_redonda_model.h5


####P7.1a descargamos el modelo usando PYRIND & URLLIB (OPCIONAL)

In [ ]:
!pip install pyprind

In [ ]:
def reporthook(count, block_size, total_size):
    global start_time
    if count == 0:
        start_time = time.time()
        return
    duration = time.time() - start_time
    progress_size = int(count * block_size)
    speed = progress_size / (1024.**2 * duration)
    percent = count * block_size * 100. / total_size
    sys.stdout.write("\r%d%% | %d MB | %.2f MB/s | %d segundos transcurrido" %
                    (percent, progress_size / (1024.**2), speed, duration))
    sys.stdout.flush()

import urllib.request
url_github_Model='https://github.com/2HenryCardenas1/DeepLearning/blob/main/Jupyter/PLN(Procesamiento_de_lenguajes_naturales)/Models/RNN_LaBibliotecaDeBabel_model.h5?raw=true'
urllib.request.urlretrieve(url_github_Model,
                           'RNN_LaBibliotecaDeBabel_model2.h5', 
                           reporthook)

100% | 60 MB | 3.53 MB/s | 17 segundos transcurrido

('RNN_LaBibliotecaDeBabel_model2.h5',
 <http.client.HTTPMessage at 0x7f35e1178c50>)

###P7.2 instanciamos el modelo descargado

In [ ]:
from tensorflow import keras

In [ ]:
new_model = keras.models.load_model('/content/el_rey_arturo_y_los_caballeros_de_la_mesa_redonda_model.h5') #Nombre del modelo descargado

In [ ]:
text = input('Ingresa cualquier texto que se te ocurra: ').lower()
print('\n\t[!]Generando texto\n')
time.sleep(1)
print(generate_text(model, start_string=text))
